In [55]:
import pandas as pd
import numpy as np


In [56]:
concrete = pd.read_excel("Concrete_Data.xls")
concrete.head()

,cement,slag,ash,water,superplastic,coarseagg,fineagg,age,strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.986111
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.887366
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.269535
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.052780
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.296075


In [57]:
concrete.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1030 entries, 0 to 1029
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   cement        1030 non-null   float64
 1   slag          1030 non-null   float64
 2   ash           1030 non-null   float64
 3   water         1030 non-null   float64
 4   superplastic  1030 non-null   float64
 5   coarseagg     1030 non-null   float64
 6   fineagg       1030 non-null   float64
 7   age           1030 non-null   int64  
 8   strength      1030 non-null   float64
dtypes: float64(8), int64(1)
memory usage: 72.5 KB


In [58]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
test = scaler.fit_transform(concrete)
test

array([[1.        , 0.        , 0.        , ..., 0.20572002, 0.07417582,
        0.9674449 ],
       [1.        , 0.        , 0.        , ..., 0.20572002, 0.07417582,
        0.7419643 ],
       [0.52625571, 0.39649416, 0.        , ..., 0.        , 0.73901099,
        0.47264169],
       ...,
       [0.10616438, 0.38786867, 0.54272864, ..., 0.46663322, 0.07417582,
        0.26617018],
       [0.1303653 , 0.51947691, 0.        , ..., 0.48896136, 0.07417582,
        0.37918535],
       [0.36278539, 0.27963272, 0.39130435, ..., 0.42022077, 0.07417582,
        0.37461561]])

In [59]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
concrete_norm = scaler.fit_transform(concrete.values)
concrete_norm = pd.DataFrame(concrete_norm, columns=concrete.columns, index = concrete.index)

from numpy import percentile
f = lambda x : percentile(x, [0, 25, 50, 75, 100])
[f(concrete_norm['strength ']), f(concrete['strength '])]


[array([0.        , 0.26630117, 0.40004982, 0.54573176, 1.        ]),
 array([ 2.33180783, 23.70711515, 34.44277358, 46.13628654, 82.5992248 ])]

In [60]:
concrete_train = concrete_norm.iloc[0:773, :]
concrete_test = concrete_norm.iloc[773:1030, :]


In [61]:
import keras
import pydot
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [62]:
# define base model
def baseline_model():
	# create model
	model = Sequential()
	model.add(Dense(1, input_dim=8, kernel_initializer='normal', activation='relu'))
	model.add(Dense(1, kernel_initializer='normal'))

	# Compile model
	model.compile(loss='mean_squared_error', optimizer='adam')
	return model

estimator = KerasRegressor(build_fn=baseline_model, epochs=100, batch_size=128, verbose=0)


In [63]:
from datetime import datetime

logdir="logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

concrete_train_X = concrete_train.iloc[:, :8]
concrete_train_y = concrete_train.iloc[:, 8]
concrete_test_X = concrete_test.iloc[:, :8]
concrete_test_y = concrete_test.iloc[:, 8]

estimator.fit(concrete_train_X, concrete_train_y, callbacks=[tensorboard_callback])


In [64]:
kfold = KFold(n_splits=10)
results = cross_val_score(estimator, concrete_train_X, concrete_train_y, cv=kfold)
print("Results: %.2f (%.2f) MSE" % (results.mean(), results.std()))


Results: -0.05 (0.03) MSE


In [65]:
from scipy.stats import pearsonr

predicted_strength = estimator.predict(concrete_test_X)

concrete_test_y = np.array(concrete_test_y)
# calculate Pearson's correlation
corr, _ = pearsonr(concrete_test_y, predicted_strength)
print('Pearsons correlation: %.3f' % corr)

test_error =  np.abs(concrete_test_y - predicted_strength)
mean_error = np.mean(test_error)
min_error = np.min(test_error)
max_error = np.max(test_error)
std_error = np.std(test_error)

print(mean_error)

Pearsons correlation: nan
0.12738075961477077


In [66]:
def larger_model():
    # create model
    model = Sequential()
    model.add(Dense(8, input_dim=8, kernel_initializer='normal', activation='relu'))
    model.add(Dense(4, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

estimator = KerasRegressor(build_fn=larger_model, epochs=100, batch_size=128, verbose=0)
estimator.fit(concrete_train_X, concrete_train_y, callbacks=[tensorboard_callback])


In [67]:
kfold = KFold(n_splits=10)
results = cross_val_score(estimator, concrete_train_X, concrete_train_y, cv=kfold)
print("Results: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Results: -0.03 (0.02) MSE
